# Dimension Reduction and Clustering

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups, load_digits
from sklearn.feature_extraction.text import TfidfVectorizer

# these are new imports for dimensionality reduction
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
# these are new imports for clustering
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram

## Dimension Reduction

### PCA

The R package [`cluster.datasets`](http://cran.r-project.org/web/packages/cluster.datasets/cluster.datasets.pdf) has some good datasets for experimenting with unsupervised learning techniques like dimensionality reduction and clustering.  Here, we'll use the `cake.ingredients.1961` dataset of cake recipes, which I've exported to a CSV.

In [ ]:
cakes = pd.read_csv("./cakes.csv")
# strip trailing whitespace in the names
cakes["Cake"] = cakes.Cake.str.strip()
cakes.head()

Let's store a dictionary of the ingredient abbreviations so we can look them up:

In [ ]:
ingredients_dict = {
    "AE": "Almond essence",
    "BM": "Buttermilk",
    "BP": "Baking powder",
    "BR": "Butter",
    "BS": "Bananas",
    "CA": "Cocoa",
    "CC": "Cottage Cheese",
    "CE": "Chocolate",
    "CI": "Crushed Ice",
    "CS": "Crumbs",
    "CT": "Cream of tartar",
    "DC": "Dried currants",
    "EG": "Eggs",
    "EY": "Egg white",
    "EW": "Egg yolk",
    "FR": "Sifted flour",
    "GN": "Gelatin",
    "HC": "Heavy cream",
    "LJ": "Lemon juice",
    "LR": "Lemon",
    "MK": "Milk",
    "NG": "Nutmeg",
    "NS": "Nuts",
    "RM": "Rum",
    "SA": "Soda",
    "SC": "Sour cream",
    "SG": "Shortening",
    "SR": "Granulated sugar",
    "SS": "Strawberries",
    "ST": "Salt",
    "VE": "Vanilla extract",
    "WR": "Water",
    "YT": "Yeast",
    "ZH": "Zwiebach"
}

Get rid of the column of cake names so that we have a numeric only dataframe:

In [ ]:
X = cakes.iloc[:, 1:]
X.head()

In [ ]:
X.shape

First, we'll run a simple PCA using the [scikit-learn class](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).  If we don't specify `n_components` or set it to `None`, it will use the maximum number of principal components:

In [ ]:
pca = PCA(n_components=None)
pca.fit(X)

Let's take a look at the explained variance of each of the principal components:

In [ ]:
pca.explained_variance_ratio_

And plot it:

In [ ]:
plt.plot(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)
plt.scatter(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)
plt.xlabel("Principal Components Number")
plt.ylabel("Percentage of Variance Explained")
plt.show()

In [ ]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
plt.plot(range(len(pca.explained_variance_ratio_)), cumsum)
plt.scatter(range(len(pca.explained_variance_ratio_)), cumsum)
plt.xlabel("Principal Components Number")
plt.ylabel("Cumulative Percentage of Variance Explained")
plt.show()

If we're looking for an "elbow", it looks like roughly 6 or 7 principal components would be enough.  To actually get each row transformed into the principal component space, we can call `transform` on an already fit `PCA` object, or we can do both at once with `fit_transform`:

In [ ]:
pca = PCA(n_components=2)
X_trans = pca.fit_transform(X)
X_trans

Next, let's define a function for plotting:

In [ ]:
def plot_PCA(pca, X, print_row_labels, row_labels, col_labels, biplot=False, y_scale=(None, None), font_size=None):
    # transform our data to PCA space
    X_trans = pca.fit_transform(X)

    # handle the scaling of the plot
    xmin, xmax = min(X_trans[:, 0]), max(X_trans[:, 0])
    if y_scale == (None, None):
        ymin, ymax = min(X_trans[:, 1]), max(X_trans[:, 1])
        xpad, ypad = 5, 5
    else:
        ymin, ymax = y_scale
        xpad, ypad = 5, 1
        
    plt.xlim(xmin - xpad, xmax + xpad)
    plt.ylim(ymin - ypad, ymax + ypad)

    # plot words instead of points
    if print_row_labels:
        for x, y, label in zip(X_trans[:, 0], X_trans[:, 1], row_labels):
            if font_size is None:
                plt.text(x, y, label)
            else:
                plt.text(x, y, label, size=font_size)
    else:
        for x, y in zip(X_trans[:, 0], X_trans[:, 1]):
            plt.scatter(x, y)
    plt.xlabel("PC 1")
    plt.ylabel("PC 2")

    # if we want a biplot, get the loading and plot
    # axes with labels
    if biplot:
        eigenvectors = pca.components_.transpose()
        for i, col in enumerate(col_labels):
            x, y = 10*eigenvectors[i][0], 10*eigenvectors[i][1]
            plt.arrow(0, 0, x, y, color='r', width=0.002, head_width=0.05)
            plt.text(x* 1.4, y * 1.4, col, color='r', ha='center', va='center')
    
    plt.show()

In [ ]:
pca = PCA(n_components=2)
plot_PCA(pca, X, True, cakes.Cake, X.columns, biplot=True)

Here, we ran PCA on a totally unscaled version of the dataset, so we see that we're influenced by two large outliers.  The first principal component is dominated by "cocoa" and "shortening" because the "One Bowl Chocolate" cake has a huge amount of these.  The second principal component is dominated by "egg whites" because of the "Angel" foodcake recipe.

In [ ]:
#cake = "One Bowl Chocolate"
cake = "Angel"
cakes[cakes.Cake==cake].transpose()

In [ ]:
#cakes.CA
#cakes.SG
cakes.EW
#cakes.EG

Let's try mean-centering the columns first:

In [ ]:
pca = PCA(n_components=2)
X_scaled = scale(X, with_mean=True, with_std=False)
plot_PCA(pca, X_scaled, True, cakes.Cake, X.columns, biplot=True)

And now both center and scale to unit variance:

In [ ]:
pca = PCA(n_components=2)
X_scaled = scale(X, with_mean=True, with_std=True)
plot_PCA(pca, X_scaled, True, cakes.Cake, X.columns, biplot=True)

Let's look at a zoomed in version:

In [ ]:
pca = PCA(n_components=2)
X_scaled = scale(X, with_mean=True, with_std=True)
plot_PCA(pca, X_scaled, True, cakes.Cake, X.columns, biplot=True, y_scale=(-1, 1))

To me, it looks like cheesecakes are off to the right on the first principal component, and the second principal component is quantifying whether the cake has fruit or not...

### t-SNE

Let's switch for a moment to the handwritten digits dataset that we saw before in the notebook on k-nearest neighbors:

In [ ]:
digits = load_digits()
digits_data = scale(digits.data)

labels = digits.target

In [ ]:
digits_data.shape

In [ ]:
pca = PCA(n_components=2)
digits_trans = pca.fit_transform(digits_data)

Let's make a plot of the first two principal components, colored and labeled by the true digit:

In [ ]:
xmin, xmax = min(digits_trans[:, 0]), max(digits_trans[:, 0])
ymin, ymax = min(digits_trans[:, 1]), max(digits_trans[:, 1])
xpad, ypad = 5, 5
plt.xlim(xmin - xpad, xmax + xpad)
plt.ylim(ymin - ypad, ymax + ypad)

for x, y, label in zip(digits_trans[:, 0], digits_trans[:, 1], labels):
    plt.text(x, y, label, size=8, color=plt.cm.Set1(label/10.))

plt.xlabel("PC 1")
plt.ylabel("PC 2")

plt.show()

If our goal is visualizing the high dimensional dataset, the [t-SNE](http://lvdmaaten.github.io/tsne/) algorithm usually does a superior job of finding structure in the high-dimensional data that can be visualized in two dimensions.  There's a [scikit-learn class](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE) for running t-SNE.

In [ ]:
tsne = TSNE(n_components=2, verbose=True)
digits_trans = tsne.fit_transform(digits_data)

In [ ]:
xmin, xmax = min(digits_trans[:, 0]), max(digits_trans[:, 0])
ymin, ymax = min(digits_trans[:, 1]), max(digits_trans[:, 1])
xpad, ypad = 5, 5
plt.xlim(xmin - xpad, xmax + xpad)
plt.ylim(ymin - ypad, ymax + ypad)

#for x, y, label in zip(digits_trans[labels==6, 0], digits_trans[labels==6, 1], labels[labels==6]):
for x, y, label in zip(digits_trans[:, 0], digits_trans[:, 1], labels):
    plt.text(x, y, label, size=8, color=plt.cm.Set1(label/10.))

plt.xlabel("Component 1")
plt.ylabel("Component 2")

plt.show()

This clearly does a better job at finding the "structure" in the high-dimensional dataset.  Notice that 3, 5, and 9 end up near each other.  But there are some 1's that are closer to the 2's, and some 9's that are closer to the 7's and the 1's.

## Clustering

### Hierarchical

Now back to cakes.  We'll use some functions from scipy to run hierarchical clustering.  `linkage` calculates the distances and linkages, and `dendrogram` displays the actual tree dendrogram:

In [ ]:
clusters_single = linkage(scale(X), method='single', metric="euclidean") # single, complete, average, and ward methods

In [ ]:
dendr = dendrogram(clusters_single, orientation="top", labels=list(cakes.Cake))

As ISLR says, single linkage tends to produce really unbalanced trees.  We can put the dendrogram on its side to make it easier to visualize:

In [ ]:
dendr = dendrogram(clusters_single, orientation="right", labels=list(cakes.Cake))

In [ ]:
clusters_complete = linkage(scale(X), method='complete', metric="euclidean") # single, complete, average, and ward methods

In [ ]:
dendr = dendrogram(clusters_complete, orientation="top", labels=list(cakes.Cake))

In [ ]:
dendr = dendrogram(clusters_complete, orientation="right", labels=list(cakes.Cake))

In [ ]:
clusters_ward = linkage(scale(X), method='ward', metric="euclidean") # single, complete, average, and ward methods

In [ ]:
dendr = dendrogram(clusters_ward, orientation="top", labels=list(cakes.Cake))

In [ ]:
dendr = dendrogram(clusters_ward, orientation="right", labels=list(cakes.Cake))

The clustering is doing something sensible: the cheesecakes group together and are on their own, the chocolate cakes are together (sour cream fudge, red devil's, sweet chocolate, and one bowl chocolate), etc.

### k-Means

As a general resource, the [scikit-learn clustering page](http://scikit-learn.org/stable/modules/clustering.html) is great.  It has all the different kinds of clustering algorithms with their pros and cons.  Here, we'll focus on k-means.

In [ ]:
# init can be k-means++ or random; k-means++ is just a smarter version of random that forces the
# centers to be further apart
kmeans = KMeans(n_clusters=10, init='k-means++', n_init=10, max_iter=300, verbose=True, n_jobs=1)

In [ ]:
kmeans.fit(digits_data)

We can see the assigned cluster or label of each data point:

In [ ]:
kmeans.labels_

And the cluster centers themselves:

In [ ]:
kmeans.cluster_centers_.shape

The "intertia" tells us the within cluster sum-of-squares, or the "sum of distances of samples to their closest cluster center."

In [ ]:
kmeans.inertia_

Here, we make a plot where we color by the k-means label instead of the true label.  We can see that things are decent, but definitely more confused than with the true labels:

In [ ]:
xmin, xmax = min(digits_trans[:, 0]), max(digits_trans[:, 0])
ymin, ymax = min(digits_trans[:, 1]), max(digits_trans[:, 1])
xpad, ypad = 5, 5
plt.xlim(xmin - xpad, xmax + xpad)
plt.ylim(ymin - ypad, ymax + ypad)

for x, y, true_label, kmeans_label in zip(digits_trans[:, 0], digits_trans[:, 1], labels, kmeans.labels_):
    plt.text(x, y, true_label, size=8, color=plt.cm.Set1(kmeans_label/10.))

plt.xlabel("Component 1")
plt.ylabel("Component 2")

plt.show()

We can call the `predict` method, which will tell us which cluster center some new data is closest too:

In [ ]:
kmeans.predict(digits_data)

The `transform` method will transform data into the cluster distance space.  That is, how far the point is from each cluster center:

In [ ]:
transformed = kmeans.transform(digits_data)
transformed[0, :]

For very large datasets, there's a much faster implementation of k-means called [mini-batch k-means](http://www.eecs.tufts.edu/~dsculley/papers/fastkmeans.pdf), and a [scikit-learn class](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) for running it:

In [ ]:
mb_kmeans = MiniBatchKMeans(n_clusters=10, batch_size=100, init='k-means++', n_init=10, max_iter=300, verbose=True)

In [ ]:
mb_kmeans.fit(digits_data)

In [ ]:
mb_kmeans.labels_

In [ ]:
xmin, xmax = min(digits_trans[:, 0]), max(digits_trans[:, 0])
ymin, ymax = min(digits_trans[:, 1]), max(digits_trans[:, 1])
xpad, ypad = 5, 5
plt.xlim(xmin - xpad, xmax + xpad)
plt.ylim(ymin - ypad, ymax + ypad)

for x, y, true_label, kmeans_label in zip(digits_trans[:, 0], digits_trans[:, 1], labels, mb_kmeans.labels_):
    plt.text(x, y, true_label, size=8, color=plt.cm.Set1(kmeans_label/10.))

plt.xlabel("Component 1")
plt.ylabel("Component 2")

plt.show()

Let's see if we can re-cover the "correct" number of clusters using the silhouette statistic:

In [ ]:
n_clusters = range(3, 70, 2)
silhouette_stats = []
for this_n_clusters in n_clusters:
    print "Fitting %s clusters..." % this_n_clusters
    kmeans = KMeans(n_clusters=this_n_clusters, init='k-means++', n_init=10, max_iter=300, verbose=False, n_jobs=1)
    kmeans.fit(digits_data)
    labels = kmeans.labels_
    silhouette_stats.append(silhouette_score(digits_data, labels, metric='euclidean'))

In [ ]:
plt.plot(n_clusters, silhouette_stats)
plt.show()

## Text Data

In this example, we'll do dimension reduction and clustering on some text data--the 20 newsgroups dataset from last week:

In [ ]:
data_train = fetch_20newsgroups(subset='train', categories=None,
                                shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

In [ ]:
data_train.target_names

Let's only keep the 'rec.sport.baseball', 'rec.autos', 'sci.space', and 'talk.politics.guns' categories:

In [ ]:
to_keep = np.where([name in [9, 7, 14, 16] for name in data_train.target])[0]
to_keep

We turn the blobs of text into numeric features by using tfidf, which is basically a normalized version of the word counts:

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=25, stop_words='english', use_idf=True)
X_train = vectorizer.fit_transform(np.array(data_train.data)[to_keep])
targets = data_train.target[to_keep]

Our feature matrix has 2,213 postings and 1,107 features, in sparse matrix format:

In [ ]:
X_train

For sparse matrices, the `TruncatedSVD` class will perform PCA much, much faster than the regular `PCA` class:

In [ ]:
svd = TruncatedSVD(n_components=500)

In [ ]:
X_train_trans = svd.fit_transform(X_train)
X_train_trans.shape

In [ ]:
svd.explained_variance_ratio_.sum()

Let's look for four clusters in the reduced dimensionality space:

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=4, init_size=1000, batch_size=1000, init='k-means++', n_init=500, max_iter=1000)
#kmeans = KMeans(n_clusters=4, init='k-means++', n_init=100, max_iter=1000)
kmeans.fit(X_train)

In [ ]:
kmeans.labels_

In [ ]:
data_train.target

In [ ]:
pd.crosstab(index=targets, columns=kmeans.labels_, rownames=['True'], colnames=['Predicted'])

Let's look at the cluster centers, and find the top 10 "directions" or terms that correspond to each:

In [ ]:
centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(4):
    print "Cluster %d:" % i
    for ind in centroids[i, :10]:
        print ' %s' % terms[ind]